## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file.
#file_name = os.path.join("Weather_Database","WeatherPy_database.csv")
#city_data_df =  pd.read_csv(file_name)
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()                         


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bambous Virieux,MU,-20.3428,57.7575,81.00,89,40,10.36,light rain
1,1,Ushuaia,AR,-54.8000,-68.3000,48.20,71,75,28.77,broken clouds
2,2,Faanui,PF,-16.4833,-151.7500,83.01,68,0,5.55,clear sky
3,3,Mahasamund,IN,21.1000,82.1000,73.40,64,40,5.75,haze
4,4,Port Elizabeth,ZA,-33.9180,25.5701,64.40,82,75,13.80,broken clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bambous Virieux,MU,-20.3428,57.7575,81.00,89,40,10.36,light rain
2,2,Faanui,PF,-16.4833,-151.7500,83.01,68,0,5.55,clear sky
3,3,Mahasamund,IN,21.1000,82.1000,73.40,64,40,5.75,haze
6,6,Rikitea,PF,-23.1203,-134.9692,79.97,73,88,4.47,light rain
13,13,Boca Do Acre,BR,-8.7522,-67.3978,70.95,98,95,2.24,overcast clouds
16,16,Hilo,US,19.7297,-155.0900,71.60,78,90,4.61,light rain
20,20,Butaritari,KI,3.0707,172.7902,81.95,79,33,17.29,light rain
23,23,Gboko,NG,7.3179,8.9951,77.00,73,100,5.86,overcast clouds
25,25,Hithadhoo,MV,-0.6000,73.0833,82.96,74,95,9.93,overcast clouds
26,26,Mabaruma,GY,8.2000,-59.7833,70.66,91,93,2.89,overcast clouds


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 202 entries, 0 to 688
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              202 non-null    int64  
 1   City                 202 non-null    object 
 2   Country              202 non-null    object 
 3   Lat                  202 non-null    float64
 4   Lng                  202 non-null    float64
 5   Max Temp             202 non-null    float64
 6   Humidity             202 non-null    int64  
 7   Cloudiness           202 non-null    int64  
 8   Wind Speed           202 non-null    float64
 9   Current Description  202 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 17.4+ KB


In [112]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bambous Virieux,MU,-20.3428,57.7575,81.00,89,40,10.36,light rain
2,2,Faanui,PF,-16.4833,-151.7500,83.01,68,0,5.55,clear sky
3,3,Mahasamund,IN,21.1000,82.1000,73.40,64,40,5.75,haze
6,6,Rikitea,PF,-23.1203,-134.9692,79.97,73,88,4.47,light rain
13,13,Boca Do Acre,BR,-8.7522,-67.3978,70.95,98,95,2.24,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
670,670,Finschhafen,PG,-6.6024,147.8681,84.51,73,61,3.62,broken clouds
678,678,Dwarka,IN,22.2394,68.9678,78.94,69,0,7.65,clear sky
679,679,Tefe,BR,-3.3542,-64.7114,70.99,98,100,1.36,overcast clouds
682,682,Hounde,BF,11.5000,-3.5167,81.12,45,68,7.74,broken clouds


In [121]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
len(hotel_df)

202

In [122]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [134]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = clean_hotel_df.replace("Nan","NaN")
clean_hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bambous Virieux,MU,81.00,light rain,-20.3428,57.7575,Casa Tia Villa
2,Faanui,PF,83.01,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
3,Mahasamund,IN,73.40,haze,21.1000,82.1000,mauhari bhantha mahasamund
6,Rikitea,PF,79.97,light rain,-23.1203,-134.9692,Pension Maro'i
13,Boca Do Acre,BR,70.95,overcast clouds,-8.7522,-67.3978,Hotel Sao Pedro
16,Hilo,US,71.60,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
20,Butaritari,KI,81.95,light rain,3.0707,172.7902,Isles Sunset Lodge
23,Gboko,NG,77.00,overcast clouds,7.3179,8.9951,Bays Garden Hotel
25,Hithadhoo,MV,82.96,overcast clouds,-0.6000,73.0833,Scoop Guest House
26,Mabaruma,GY,70.66,overcast clouds,8.2000,-59.7833,NaN


In [135]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [141]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [142]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))